In [2]:
# Q learning practice
# guide https://towardsdatascience.com/simple-reinforcement-learning-q-learning-fcddc4b6fe56
# guide 2 https://itnext.io/reinforcement-learning-with-q-tables-5f11168862c8
# copy to test the q table functions
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation

ModuleNotFoundError: No module named 'numpy'

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Load in maze from file
def load_maze(mazename):
    return np.load(mazename)

maze = load_maze("maze3by3.npy")


# Boundary Setting
L = 3 # length
win_x, win_y = 2,2

# Q_table creation
# right:0 left:1 down:2 up:3
q_table = np.zeros((L,L,4), int)

for i in range(L):
    for j in range(L):
        if j == 0: # up // left on 2d array
            q_table[i,j,1] = -100
            print(q_table[i,j])
        if i == 0: # right // up on 2d
            q_table[i,j,3] = -100
            print(q_table[i,j])
        if j == L-1: # left // right
            q_table[i,j,0] = -100
            print(q_table[i,j])
        if i == L-1: # right // down
            q_table[i,j,2] = -100
            print(q_table[i,j])
        print(maze[i,j])
            
        # for obstacle placement, we treat them as badly as wall boundaries
        if maze[i,j] == -100:
            print('obstacle at',(i,j))
            # boundary checker
            if i > 1: # down
                q_table[i-1,j,2] = -100
                
            if i < L-1:# up
                q_table[i+1,j,3] = -100
                
            if j > 1: # right
                q_table[i,j-1,0] = -100
                
            if j < L-1: #left
                q_table[i,j+1,1] = -100
                

# for winning spot
q_table[win_x-1,win_y,2] = 100
#q_table[win_x+1,win_y,3] = 100
q_table[win_x,win_y-1,0] = 100
#q_table[win_x,win_y+1,1] = 100

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# define class of agent
class agent():
    # Default Constructor
    def __init__(self, name, pos_x = 0, pos_y = 0):
        self.name = name
        self.position_x = pos_x
        self.position_y = pos_y
        print("agent created:",name)
        
    def curr_position(self):
        return (self.position_x,self.position_y)
    
    def set_curr_position(self,new_x,new_y):
        self.position_x = new_x
        self.position_y = new_y
        
    def get_x(self):
        return self.position_x
    
    def get_y(self):
        return self.position_y
    
        
    # Movement method for our agent
    def movement(self, maze, move):
        #view_maze(maze)
        print("new movement made",move)
        
        # go right
        if move == 0:
            self.position_y+=1
            print("right")
        # go left
        elif move == 1:
            self.position_y-=1
            print("left")
        # move down
        elif move == 2:
            self.position_x+=1
            print("down")
        # move up
        elif move == 3:
            self.position_x-=1
            print("up")
        else:
            print("stay still")
            
p1 = agent("Agent 1", 0,0)

In [ ]:
def getAllPossibleMove(cur_x, cur_y):
    # list comprehension check on positioning
    poss_step = [x != -100 for x in q_table[cur_x,cur_y]]
    action = []
    #print(poss_step)
    if(poss_step[0]):
        action.append(0)
    if(poss_step[1]):
        action.append(1)
    if(poss_step[2]):
        action.append(2)
    if(poss_step[3]):
        action.append(3)
    #print('a',action)
    return action


def peek_next(choice, maze):
    print('yo')

def getWinningMove(cur_x, cur_y, q_matrix):
    poss_step = [x != -100 for x in q_table[cur_x,cur_y]]
    
    print(poss_step)
    max_val = max(poss_step)
    #print('max value: ',max_val)
    max_index = np.argmax(max_val)
    #print('index:',max_index)
    return max_index




def CheckGameOver(cur_x,cur_y, maze, win_x, win_y):
    if cur_x == win_x and cur_y == win_y:
        print("win")
        return True
    elif maze[cur_x,cur_y] == -100:
        print("Death")
        return True
    elif cur_x == L or cur_y == L or cur_x == -1 or cur_y == -1:
        print("fell out of bounds")
        return True
    else:
        print("its okay")
        return False

In [ ]:
discount = 0.9
learning_rate = 0.1
epsilon = 0.5 # ratio at which the agent will explore vs exploit
maze_frames = list()

for episode in range(100):
    # reset the maze
    print('start of new episode -----------------')
    maze = load_maze("maze3by3.npy")
    # start off at our random position
    cur_x = random.randint(0,L-1)
    cur_y = random.randint(0,L-1)
    
    print("random x y",cur_x,cur_y)
    p1.set_curr_position(cur_x, cur_y)
    # visual_maze = copy.deepcopy(maze) # its own copy
    visual_maze = load_maze("maze3by3.npy")
    visual_maze[win_x,win_y] = 100
    
    
    visual_maze[cur_x,cur_y] = 1000
    while(not (CheckGameOver(cur_x, cur_y, maze, win_x, win_y))):
        poss_act = getAllPossibleMove(cur_x, cur_y)
        if random.uniform(0,1) < epsilon:
            # Explore: select a random action
            print("exploring..")
            choice = random.choice(poss_act)
        else:
            # Exploit: select action with max value
            print("exploiting..")
            choice = getWinningMove(cur_x, cur_y, q_table)
        p1.movement(maze,choice)
        cur_x_1 = p1.get_x()
        cur_y_1 = p1.get_y()
        
        if cur_x_1 == L or cur_x_1 < 0 or cur_y_1 == L or cur_y_1 < 0:
            print('went to outer bounds')
            break
        
        #print(q_table[cur_x_1,cur_y_1,choice])
        disc = discount *max(q_table[cur_x_1,cur_y_1])-q_table[cur_x,cur_y,choice]
        #print('d',disc)
        stuff = q_table[cur_x, cur_y, choice]+learning_rate*(maze[cur_x,cur_y])
        #print('s',stuff)
        q_table[cur_x,cur_y,choice] = disc + stuff
        
        # movement on the board
        visual_maze[cur_x,cur_y] = 0
        visual_maze[cur_x_1,cur_y_1] = 1000
        print(visual_maze)
        maze_frames.append(visual_maze)
        plt.imshow(visual_maze)
        plt.colorbar()
        plt.show()
        cur_x = cur_x_1
        cur_y = cur_y_1
        #q_table[cur_x,cur_y,choice] = q_table[cur_x, cur_y, choice]+learning_rate*(maze[cur_x,cur_y] + discount *max(q_table[cur_x_1,cur_y_1]-q_table[cur_x,cur_y,choice]))
    else:
        print("spawned on goal or death")
    print('----------------end spot----------------')
    #plt.imshow(visual_maze)
    #plt.show()
    visual_maze[cur_x,cur_y] = 0
    maze_frames.append(visual_maze)
    print("Episode",episode,"done ---------------------------------")
print("Training Done")


In [ ]:
maze

In [ ]:
q_table


In [ ]:
maze

In [ ]:

# only picks best path



for episode in range(20):
    # reset the maze
    maze = load_maze("maze3by3.npy")
    # start off at our random position
    cur_x = random.randint(0,L-1)
    cur_y = random.randint(0,L-1)
    print("random x y",cur_x,cur_y)
    p1.set_curr_position(cur_x, cur_y)
    visual_maze = load_maze("maze3by3.npy") # its own copy
    visual_maze[win_x,win_y] = 100000
    visual_maze[cur_x,cur_y] = 100
    #print(visual_maze)
    
    while(not (CheckGameOver(cur_x, cur_y, maze, win_x, win_y))):
        print('ayo')
        visual_maze[cur_x,cur_y] = 100
        print(visual_maze)
        
        choice = getWinningMove(cur_x, cur_y, q_table)
        cur_x_1 = p1.get_x()
        cur_y_1 = p1.get_y()
        if cur_x_1 == L or cur_x_1 < 0 or cur_y_1 == L or cur_y_1 < 0:
            print('broken')
            break
        
        p1.movement(maze,choice)
        #print(q_table[cur_x_1,cur_y_1,choice])
        
        print('choice,',choice)
        #print(q_table[cur_x_1,cur_y_1,choice])
        disc = discount *max(q_table[cur_x_1,cur_y_1])-q_table[cur_x,cur_y,choice]
        print('d',disc)
        stuff = q_table[cur_x, cur_y, choice]+learning_rate*(maze[cur_x,cur_y])
        print('s',stuff)
        q_table[cur_x,cur_y,choice] = disc + stuff
        
        # movement on the board
        visual_maze[cur_x,cur_y] = 0
        visual_maze[cur_x_1,cur_y_1] = 100
        print(visual_maze)
        plt.imshow(visual_maze)
        cur_x = cur_x_1
        cur_y = cur_y_1
        print('neyo')
    print("Episode",episode,"done ---------------------------------")
print("Training Done")



In [ ]:
anim = []
fig1 = plt.figure(figsize=(8,8))
ax = fig1.add_subplot(111)
print(len(maze_frames))
for n in range(0,200):
    anim.append([ax.matshow(maze_frames[n])])
im = animation.ArtistAnimation(fig1, anim, interval = 1000)

plt.rcParams['animation.html'] = 'jshtml'
im


In [ ]:
for i in range(50):
    plt.imshow(maze_frames[i])
    plt.show()

In [ ]:
getWinningMove(0, 0, q_table)

In [ ]:
count = 0
print("                   right left down up")
for i in range(L):
    for j in range(L):
        count+=1
        print(count, 'coords:',(i,j),q_table[i,j])